In [143]:
import json

In [152]:
with open('./profile-www.cryptonoter.me.json') as f:
    raw = json.load(f)

In [153]:
profile = raw['profile']
profile.keys()

dict_keys(['nodes', 'samples', 'timeDeltas', 'startTime', 'endTime'])

In [154]:
import numpy as np

# Unit: microseconds
np.average(profile['timeDeltas']), np.std(profile['timeDeltas'])

(198.75716908608186, 318.72477710263485)

In [155]:
# This should be equivalent to "sorting by self time" in chrome gui
sorted(profile['nodes'], key=lambda x: x['hitCount'], reverse=True)[:4]

[{'callFrame': {'columnNumber': -1,
   'functionName': '(idle)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 299699,
  'id': 4},
 {'callFrame': {'columnNumber': -1,
   'functionName': '(program)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 2520,
  'id': 2},
 {'callFrame': {'columnNumber': 29277,
   'functionName': '',
   'lineNumber': 0,
   'scriptId': '19',
   'url': ''},
  'children': [278, 280, 285, 289, 295, 301, 315, 316],
  'hitCount': 123,
  'id': 272,
  'positionTicks': [{'line': 1, 'ticks': 123}]},
 {'callFrame': {'columnNumber': 25413,
   'functionName': '_0x2d0c.lbeIEt',
   'lineNumber': 0,
   'scriptId': '16',
   'url': 'https://www.cryptonoter.me/prototyper-show.js'},
  'children': [11, 12, 13, 14, 15, 16, 17, 18, 19],
  'hitCount': 63,
  'id': 10,
  'positionTicks': [{'line': 1, 'ticks': 63}]}]

In [156]:
# This should be equivalent to "Call tree" in chrome gui

result = {}
def calc(node_id):
    cached = result.get(node_id)
    if cached:
        return cached
    
    node = profile['nodes'][node_id - 1]
    child = node.get('children', [])
    ans = node['hitCount']
    for cid in child:
        ans += calc(cid)
    
    result[node_id] = ans
    return ans

calc(1)
call_tree = sorted([(k,v,profile['nodes'][k-1])for k,v in result.items()], key=lambda x: x[1], reverse=True)[:50]
call_tree

[(1,
  302842,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(root)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'children': [2,
    3,
    4,
    5,
    48,
    49,
    76,
    123,
    134,
    140,
    245,
    262,
    264,
    265,
    272,
    273,
    276,
    277,
    283],
   'hitCount': 0,
   'id': 1}),
 (4,
  299699,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(idle)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'hitCount': 299699,
   'id': 4}),
 (2,
  2520,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(program)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'hitCount': 2520,
   'id': 2}),
 (5,
  147,
  {'callFrame': {'columnNumber': 0,
    'functionName': '',
    'lineNumber': 0,
    'scriptId': '16',
    'url': 'https://www.cryptonoter.me/prototyper-show.js'},
   'children': [6, 8, 20, 30],
   'hitCount': 12,
   'id': 5,
   'positionTicks': [{'line': 1, 'ticks': 12}]}),
 (273,
  14

limit_output extension: Maximum message size of 5000 exceeded with 14434 characters

In [157]:
def getAllChildren(cid):
    node = profile['nodes'][cid-1]
    child = node.get('children', [])
    ans = list(child)
    for x in child:
        ans += getAllChildren(x)
    return ans

most_called = next(x for x in call_tree if '(' not in x[2]['callFrame']['functionName'])
print(most_called)
print(max([(result[x], profile['nodes'][x-1]['callFrame']) for x in getAllChildren(most_called[0])], key=lambda x: x[0]))

(5, 147, {'positionTicks': [{'line': 1, 'ticks': 12}], 'hitCount': 12, 'children': [6, 8, 20, 30], 'id': 5, 'callFrame': {'url': 'https://www.cryptonoter.me/prototyper-show.js', 'columnNumber': 0, 'lineNumber': 0, 'scriptId': '16', 'functionName': ''}})
(99, {'url': 'https://www.cryptonoter.me/prototyper-show.js', 'columnNumber': 24543, 'lineNumber': 0, 'scriptId': '16', 'functionName': '_0x2d0c'})


In [158]:
# But what's their actual runtime? We only know hit counts.
# Estimate through profile['timeDeltas']:
child = getAllChildren(most_called[0]) + [most_called[0]]
sum([profile['timeDeltas'][i] for i, x in enumerate(profile['samples']) if x in child])

22732

In [159]:
# How much time is this, compared to "(root) - (idle)" ?
root = next(n for n in profile['nodes'] if n['callFrame']['functionName'] == '(root)')
idle = next(n for n in profile['nodes'] if n['callFrame']['functionName'] == '(idle)')
print((result[root['id']] - result[idle['id']]))

3143
